In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('tracks.csv', skiprows=1, index_col=0, engine='python', error_bad_lines=False)

Skipping line 53396: unexpected end of data


In [4]:
tracks = df.copy()

In [5]:
tracks = tracks.dropna(subset=['genre_top'])

In [6]:
import category_encoders as ce

cat_encoder = ce.OrdinalEncoder(return_df=False)
tracks['genre_top_encoded'] = cat_encoder.fit_transform(tracks['genre_top'])

In [7]:
X = tracks.drop(columns=['genre_top', 'genre_top_encoded', 'id', 'id.1', 'genres', 'genres_all'], axis=1)
y = tracks['genre_top_encoded']
X.shape, y.shape

((28426, 47), (28426,))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
X_train.columns

Index(['comments', 'date_created', 'date_released', 'engineer', 'favorites',
       'information', 'listens', 'producer', 'tags', 'title', 'tracks', 'type',
       'active_year_begin', 'active_year_end', 'associated_labels', 'bio',
       'comments.1', 'date_created.1', 'favorites.1', 'latitude', 'location',
       'longitude', 'members', 'name', 'related_projects', 'tags.1', 'website',
       'wikipedia_page', 'split', 'subset', 'bit_rate', 'comments.2',
       'composer', 'date_created.2', 'date_recorded', 'duration',
       'favorites.2', 'information.1', 'interest', 'language_code', 'license',
       'listens.1', 'lyricist', 'number', 'publisher', 'tags.2', 'title.1'],
      dtype='object')

In [14]:
X_train.shape

(22740, 47)

In [20]:
hash_features = ['producer', 'location', 'composer']
X_train_features = pd.DataFrame()
X_test_features = pd.DataFrame()
for hash_feature in hash_features:
    hasher = ce.HashingEncoder(n_components=10)
    hasher = hasher.fit(X_train[hash_feature])
    hasher_train = hasher.transform(X_train.loc[:,hash_feature])
    hasher_test = hasher.transform(X_test.loc[:,hash_feature])
    X_train_features = pd.concat([X_train_features, hasher_train], axis=1)
    X_test_features = pd.concat([X_test_features, hasher_test], axis=1)

In [22]:
X_train_features.shape, X_test_features.shape

((22740, 30), (5686, 30))